In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from tqdm import tqdm
import numpy as np
import tensorflow as tf
import os
import pickle
import json
import re
from PIL import Image

In [2]:
tf.test.is_gpu_available()

True

In [3]:
# create giant dictionary for all data
data_dir = '../facebook_challenge_data/'
model_dir = 'models/'

# load data and print sizes
def get_dict(path):
    jsonl_content = open(path, 'r').read()
    data = [json.loads(jline) for jline in jsonl_content.split('\n')]
    return {datum['id'] : datum for datum in data}


train_dict = get_dict(data_dir + 'train.jsonl')
val_dict = get_dict(data_dir + 'dev.jsonl')
test_dict = get_dict(data_dir + 'test.jsonl')

print(len(train_dict))
print(len(val_dict))
print(len(test_dict))

8500
500
1000


In [4]:
from random import randint # for random cropping
from tensorflow.keras.preprocessing.sequence import pad_sequences

class FBMMDataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data_dict, tokenizer, pad_len, batch_size=32, dim=(299, 299), n_channels=3, shuffle=True):
        'Initialization'
        self.dim = dim
        self.data_dict = data_dict
        self.batch_size = batch_size
        self.n_channels = n_channels
        self.shuffle = shuffle
        self.pad_len = pad_len
        self.tokenizer = tokenizer
        
        # build labels list and id list
        self.id_list = list(self.data_dict.keys())
        self.labels = {ID: self.data_dict[ID]['label'] for ID in self.id_list}
        self.img_list = {ID: self.data_dict[ID]['img'] for ID in self.id_list}
            
        # get text dictionary
        self.text_dict = self.process_text(self.id_list)
        
        self.on_epoch_end()
        self.classes = [self.labels[self.id_list[i]] for i in self.indexes]

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.id_list) / self.batch_size)) + 1 # last batch is partial

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:index*self.batch_size + self.batch_size]
        
        
        # Find list of IDs
        id_list_temp = [self.id_list[k] for k in indexes]

        # Generate data
        X_txt, X_img, y = self.__data_generation(id_list_temp)
        
        return (X_txt, X_img), y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.id_list))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, id_list_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X_img = np.empty((len(id_list_temp), *self.dim, self.n_channels))
        X_txt = np.empty((len(id_list_temp), self.pad_len))
        y = np.empty(len(id_list_temp), dtype=int)

        # Generate data
        for i, ID in enumerate(id_list_temp):
            # Store sample
            X_img[i,] = self.process_img(data_dir + self.img_list[ID])
            X_txt[i,] = self.text_dict[ID]

            # Store class
            y[i] = self.labels[ID]

        return X_txt.astype(int), X_img, y
    
    def process_img(self, path): # method for getting image
        img = Image.open(path)
        img.load()
        img = img.resize(self.dim, Image.ANTIALIAS)
        data = np.asarray(img, dtype='uint8')
        im = self.augment(data)
        
        
        if im.shape==(self.dim[0], self.dim[1]): im = np.stack((im,)*3, axis=-1) # handle grayscale
        if im.shape == (*self.dim, 4): im = im[:,:,:3] # handle weird case
        
        return im
    
    def augment(self, im): # random crop and random mirror
        
        # random crop
        x_max, y_max = im.shape[0], im.shape[1]
        x_start, y_start = randint(0, x_max - self.dim[0]), randint(0, y_max - self.dim[1])
        im = im[x_start:x_start + self.dim[0], y_start:y_start + self.dim[1]]
        
        # random mirror
        if randint(0,1): im = np.flip(im, axis=1)
        
        return im
    
    def process_text(self, id_list):
        
        # matrix for texts
        texts = [self.data_dict[ID]['text'] for ID in id_list]
        sequences = [self.tokenizer.encode(text) for text in texts] # make this more efficient...
        text_seqs = pad_sequences(sequences, maxlen=self.pad_len)
        
        id_to_seq = {ID: txt for (ID, txt) in zip(id_list, text_seqs)} # map id to text seq
        
        return id_to_seq

In [5]:
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, TFRobertaModel
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
max_len = 50

# create data generators
train_gen = FBMMDataGenerator(data_dict=train_dict,
                          tokenizer=tokenizer,
                          pad_len=max_len,
                          batch_size=16,
                          dim=(224, 224),
                          n_channels=3,
                          shuffle=True)

val_gen = FBMMDataGenerator(data_dict=val_dict,
                          tokenizer=tokenizer,
                          pad_len=max_len,
                          batch_size=16,
                          dim=(224, 224),
                          n_channels=3,
                          shuffle=True)

# Define model

In [6]:
# # load resnet
# resnet = tf.keras.applications.resnet.ResNet152(include_top=False, 
#                                                 weights='imagenet', 
#                                                 input_shape=(224, 224, 3))

In [7]:
# # tf.config.experimental_run_functions_eagerly(True) # https://github.com/tensorflow/tensorflow/issues/34944MAX_SEQ_LEN = 50 # 50 words
# from tensorflow.keras import backend as K

# MAX_SEQ_LEN=50

# # load pretrained roberta
# import transformers
# from tensorflow.keras import layers

# # load roberta
# class ROBERTA(transformers.TFRobertaModel):
#     def __init__(self, config, *inputs, **kwargs):
#         super(ROBERTA, self).__init__(config, *inputs, **kwargs)
#         self.roberta.call = tf.function(self.roberta.call)
# roberta = ROBERTA.from_pretrained('roberta-base')

# # get embedding matrix
# text_embedding = roberta.get_input_embeddings().word_embeddings
# print(type(text_embedding))

# # handle text
# input_ids = layers.Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32)
# text_embeds = tf.gather(text_embedding, input_ids)
# print(type(text_embeds))

# # handle images
# input_imgs = layers.Input(shape=(224,224,3))
# img_embeds = resnet(input_imgs)
# img_embeds = layers.AveragePooling2D(pool_size=(2, 2))(img_embeds)
# print('Image embedding shape after pooling (K, M):', img_embeds.shape)
# img_embeds = layers.Reshape((img_embeds.shape[1] * img_embeds.shape[2], img_embeds.shape[3]))(img_embeds) # KM*2048
# print('Image embedding shape after reshape:', img_embeds.shape)
# img_embeds = layers.Dense(768)(img_embeds)

# # combine and get token type ids
# print('Text and image embedding shapes:', text_embeds.shape, img_embeds.shape)
# combined_embed = layers.Concatenate(axis=1)([text_embeds, img_embeds])
# seg_ids = np.concatenate((np.zeros(MAX_SEQ_LEN, dtype=np.int8), np.ones(img_embeds.shape[1], dtype=np.int8)))

# # pass through roberta
# roberta_encodings = roberta(inputs={'inputs_embeds': combined_embed, 
#                             'attention_mask': None,
#                              'token_type_ids': None})[0]#, token_type_ids=seg_ids)[0]
# doc_encoding = tf.squeeze(roberta_encodings[:, 0:1, :], axis=1) # Keep [CLS] token encoding
# doc_encoding = layers.Dropout(0.1)(doc_encoding) # Apply dropout
# outputs = layers.Dense(1, activation='sigmoid', name='outputs')(doc_encoding)
# model = tf.keras.models.Model(inputs=[input_ids, input_imgs], outputs=[outputs])

# # compile
# optimizer = tf.keras.optimizers.Adam(lr=5e-6)
# model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# model.summary()

In [8]:
import transformers
from tensorflow.keras import layers

class ROBERTA(transformers.TFRobertaModel):
    def __init__(self, config, *inputs, **kwargs):
        super(ROBERTA, self).__init__(config, *inputs, **kwargs)
        self.roberta.call = tf.function(self.roberta.call)


class MyModel(tf.keras.Model):
    
    def __init__(self):
        super(MyModel, self).__init__()
        self.resnet = tf.keras.applications.resnet.ResNet152(include_top=False, 
                                                             weights='imagenet', 
                                                             input_shape=(224, 224, 3))
        self.roberta = ROBERTA.from_pretrained('roberta-base')
        
        self.text_embedding = self.roberta.get_input_embeddings().word_embeddings
         
        self.pooling = layers.AveragePooling2D(pool_size=(2, 2))
        self.reshape = layers.Reshape((3 * 3, 2048))
        self.dense1 = layers.Dense(768)
        
        self.concat = layers.Concatenate(axis=1)
        
        self.denseout = layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        text, image = inputs
        
#         print(text, image)
        
        image_emb = self.resnet(image)
        image_emb = self.pooling(image_emb)
        image_emb = self.reshape(image_emb)
        image_emb = self.dense1(image_emb)
        
        text_emb = tf.gather(self.text_embedding, text)
        
        concat_emb = self.concat([text_emb, image_emb])
        roberta_encodings = self.roberta(inputs={'inputs_embeds': concat_emb})[0]
#         print(type(roberta_encodings))
        doc_encoding = tf.squeeze(roberta_encodings[:, 0:1, :], axis=1)
        
        output = self.denseout(doc_encoding)
        
        return output

model = MyModel()

In [9]:
optimizer = tf.keras.optimizers.Adam(lr=5e-6)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# train model
from tensorflow.keras.callbacks import ModelCheckpoint

mcp_save = ModelCheckpoint(model_dir + 'best_roberta_bt_weights.h5', 
                           save_weights_only=True, 
                           save_best_only=True, 
                           verbose=1,
                           monitor='val_loss', 
                           mode='min')

history = model.fit_generator(train_gen,
                    validation_data=val_gen,
                    shuffle=True,
                    epochs=1,
                    callbacks=[mcp_save])

    def call(self, inputs):
        text, image = inputs
        
#         print(text, image)
        
        image_emb = self.resnet(image)
        image_emb = self.pooling(image_emb)
        image_emb = self.reshape(image_emb)
        image_emb = self.dense1(image_emb)
        
        text_emb = tf.gather(self.text_embedding, text)
        
        concat_emb = self.concat([text_emb, image_emb])
        roberta_encodings = self.roberta(inputs={'inputs_embeds': concat_emb})[0]
#         print(type(roberta_encodings))
        doc_encoding = tf.squeeze(roberta_encodings[:, 0:1, :], axis=1)
        
        output = self.denseout(doc_encoding)
        
        return output

This may be caused by multiline strings or comments not indented at the same level as the code.
    def call(self, inputs):
        text, image = inputs
        
#         print(text, image)
        
        image_emb = self.resnet(image)
        image_emb = self.pooling(image_emb)
        image_emb = self.reshape

In [ ]:
# test
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
import math

y_val = val_gen.classes

# get AUROC
preds = fcm_model.predict_generator(val_gen)
print('Val AUROC:', roc_auc_score(y_val, preds))

# get loss and acc
preds_bin = np.array(preds)
preds_bin[preds>0.5] = 1
preds_bin[preds<=0.5] = 0
print('Val Accuracy:', accuracy_score(y_val, preds_bin))